In [1]:
%run default-imports.ipynb

Using TensorFlow backend.


In [2]:
%run run-experiment.ipynb

In [3]:
# get explanation features of best performing model
# train the model on SINAI using all of the top features from the model
# remove the features one by one (recursive feature elimination) (iRFE)
# apply recursive feature elimination to current sinai data ()

In [4]:
''' define options to run experiments on '''
options = {
    'target' : ["AKI"],
    'cohort' : ['MIMIC'],
    'test_size' : [0.2],
    'imputation_method' : [imputers.DEFAULT],
    'algorithm' : [algorithms.RF],
    'sampling_method': [None],
    'scale_method': [None],
    'optimize_mode': [False],
    'num_features': [50], 
    'crossval': [True],
    'explainers' : [['MimicExplainer', 'ShapExplainer', 'FeatContribExplainer't, 'LimeExplainer']]
    #'explainers' : [['MimicExplainer', 'ShapExplainer', 'FeatContribExplainer']]
    #'explainers' : [['ShapExplainer', 'FeatContribExplainer']]
    #'explainers' : [['FeatContribExplainer']]
}


In [5]:
#experiments = unpickle('experiments.json') or {}
#exp_id = list(sorted([(k,v['performance']['discrimination']['auc']) for k,v in experiments.items()], key=lambda x: x[1], reverse=True))[:1][0][0]
#weighted_explanations = experiments[exp_id]['weighted_explanations']
#%run feature-list.ipynb
#top_features

In [6]:
rfe_experiments = unpickle('rfe_experiments.json') or {}

Could not unpickle object. [Errno 2] No such file or directory: 'rfe_experiments.json'


In [7]:
max_features = 10
number_features = 10000
previous_experiment = {}
n_experiments = 0

while number_features > max_features:
    ''' iterate over different options '''
    try:
        params = dict(zip(options.keys(), list(next(product(*options.values())))))        
        if previous_experiment:
            features_to_select = [exp[0] for exp in previous_experiment['weighted_explanations']]
            del features_to_select[0]
            print(f'Selected features: {len(features_to_select)}')
            params['features_to_select'] = features_to_select
            number_features = len(features_to_select)
            params['num_features'] = number_features
        experiment = run_experiment(params)
        exp_id = str(uuid.uuid1())
        rfe_experiments[exp_id] = experiment
        previous_experiment = experiment            
        n_experiments += 1
    except Exception as e:
        print(f"Could not run experiment with params {str(params)}. Error: {str(e)}")

Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 100, 'crossval': True, 'explainers': ['FeatContribExplainer']}
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 805.175ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 99
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 99, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['PEPTIC_ULCER', 'METASTATIC_CANCER', 'AIDS', 'SODIUM (WHOLE 

*** Training of classifier ready. Time elapsed: 784.444ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 96
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 96, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['LYMPHOMA', 'AIDS', 'PSYCHOSES', 'HEMATOCRIT (CALCULATED)_DTS_2.0', 'LACTATE_DTS_2.0', 'DEFICIENCY_ANEMIAS', 'BLOOD_LOSS_ANEMIA', 'SOLID_TUMOR', 'POTASSIUM WHOLE BLOOD_DTS_2.0', 'HEMOGLOBIN (BLOOD GAS)_DTS_2.0', 'RHEUMATOID_ARTHRITIS', 'LACTATE_DTS_3.0', 'ALCOHOL_ABUSE', 'DRUG_ABUSE', 'HEMOGLOBIN (BLOOD GAS)_DTS_3.0', 'HEMATOCRIT (CALCULATED)_DTS_3.0', 'DEPRESSION', 'POTASSIUM WHOLE BLOOD_DTS_3.0', 'PARALYSIS', 'GLUCOSE (BLOOD GAS)

*** Training of classifier ready. Time elapsed: 853.024ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 92
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 92, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['DRUG_ABUSE', 'HEMOGLOBIN (BLOOD GAS)_DTS_2.0', 'POTASSIUM WHOLE BLOOD_DTS_2.0', 'DEFICIENCY_ANEMIAS', 'HEMATOCRIT (CALCULATED)_DTS_2.0', 'LACTATE_DTS_2.0', 'GLUCOSE (BLOOD GAS)_DTS_3.0', 'HEMOGLOBIN (BLOOD GAS)_DTS_3.0', 'ALCOHOL_ABUSE', 'BLOOD_LOSS_ANEMIA', 'HEMATOCRIT (CALCULATED)_DTS_3.0', 'LACTATE_DTS_3.0', 'RHEUMATOID_ARTHRITIS', 'PARALYSIS', 'POTASSIUM WHOLE BLOOD_DTS_3.0', 'GLUCOSE (BLOOD GAS)_DTS_2.0', 'DEPRESSION', 'HYPOT

*** Training of classifier ready. Time elapsed: 982.036ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 88
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 88, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['HEMOGLOBIN (BLOOD GAS)_DTS_2.0', 'HEMATOCRIT (CALCULATED)_DTS_2.0', 'ALCOHOL_ABUSE', 'POTASSIUM WHOLE BLOOD_DTS_2.0', 'POTASSIUM WHOLE BLOOD_DTS_3.0', 'LACTATE_DTS_3.0', 'HEMATOCRIT (CALCULATED)_DTS_3.0', 'GLUCOSE (BLOOD GAS)_DTS_3.0', 'HEMOGLOBIN (BLOOD GAS)_DTS_3.0', 'RHEUMATOID_ARTHRITIS', 'DEPRESSION', 'GLUCOSE (BLOOD GAS)_DTS_2.0', 'HYPOTHYROIDISM', 'CHLORIDE (WHOLE BLOOD)_DTS_1.0', 'PARALYSIS', 'HEMATOCRIT (CALCULATED)_DTS_1

*** Training of classifier ready. Time elapsed: 957.662ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 84
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 84, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['RHEUMATOID_ARTHRITIS', 'ALCOHOL_ABUSE', 'POTASSIUM WHOLE BLOOD_DTS_3.0', 'HEMOGLOBIN (BLOOD GAS)_DTS_3.0', 'HEMATOCRIT (CALCULATED)_DTS_3.0', 'CHLORIDE (WHOLE BLOOD)_DTS_1.0', 'GLUCOSE (BLOOD GAS)_DTS_3.0', 'GLUCOSE (BLOOD GAS)_DTS_2.0', 'HEMATOCRIT (CALCULATED)_DTS_1.0', 'DEPRESSION', 'OTHER_NEUROLOGICAL', 'HYPOTHYROIDISM', 'HEMOGLOBIN (BLOOD GAS)_DTS_1.0', 'ALBUMIN_DTS_3.0', 'DIABETES_COMPLICATED', 'SODIUM (WHOLE BLOOD)_DTS_1.0'

...

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 604.393ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 42
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 42, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['SODIUM_DTS_3.0', 'SODIUM_DTS_2.0', 'PT_DTS_3.0', 'RENAL_FAILURE', 'GLUCOSE_DTS_3.0', 'CHLORIDE_DTS_3.0', 'PTT_DTS_2.0', 'WHITE BLOOD CELLS_DTS_3.0', 'CHLORIDE_DTS_2.0', 'ANION GAP_DTS_1.0', 'PTT_DTS_3.0', 'INR(PT)_DTS_1.0', 'PT_DTS_2.0', 'SODIUM_DTS_1.0', 'POTASSIUM_DTS_2.0', 'WHITE BLOOD CELLS_DTS_2.0', 'POTASSIUM_DTS_1.0', 'GLUCOSE_DTS_2.0', 'CHLORIDE_DTS_1.0', 'HEMATOCRIT_

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 757.241ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 36
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 36, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['CHLORIDE_DTS_3.0', 'WHITE BLOOD CELLS_DTS_3.0', 'WHITE BLOOD CELLS_DTS_2.0', 'ANION GAP_DTS_1.0', 'INR(PT)_DTS_1.0', 'GLUCOSE_DTS_3.0', 'SODIUM_DTS_1.0', 'POTASSIUM_DTS_2.0', 'PLATELET COUNT_DTS_3.0', 'CHLORIDE_DTS_2.0', 'PT_DTS_2.0', 'POTASSIUM_DTS_1.0', 'CHLORIDE_DTS_1.0', 'GLUCOSE_DTS_2.0', 'GLUCOSE_DTS_1.0', 'HEMATOCRIT_DTS_2.0', 'LIVER_DISEASE', 'PLATELET COUNT_DTS_2.0',

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 857.054ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 29
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 29, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['INR(PT)_DTS_1.0', 'SODIUM_DTS_1.0', 'LIVER_DISEASE', 'WHITE BLOOD CELLS_DTS_2.0', 'CHLORIDE_DTS_1.0', 'HEMOGLOBIN_DTS_1.0', 'HEMATOCRIT_DTS_2.0', 'HEMATOCRIT_DTS_1.0', 'GLUCOSE_DTS_2.0', 'POTASSIUM_DTS_1.0', 'PLATELET COUNT_DTS_3.0', 'HEMOGLOBIN_DTS_3.0', 'PLATELET COUNT_DTS_2.0', 'WHITE BLOOD CELLS_DTS_1.0', 'HEMATOCRIT_DTS_3.0', 'PT_DTS_1.0', 'PLATELET COUNT_DTS_1.0', 'HEMO

*** Obtaining feature importances via classifier:
Selected features: 22
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 22, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['GLUCOSE_DTS_2.0', 'CHLORIDE_DTS_1.0', 'HEMOGLOBIN_DTS_1.0', 'HEMATOCRIT_DTS_1.0', 'HEMOGLOBIN_DTS_3.0', 'FLUID_ELECTROLYTE', 'PLATELET COUNT_DTS_2.0', 'HEMATOCRIT_DTS_3.0', 'BLD CREA_DTS_3.0', 'PLATELET COUNT_DTS_1.0', 'WHITE BLOOD CELLS_DTS_1.0', 'PTT_DTS_1.0', 'GLUCOSE_DTS_1.0', 'HEMOGLOBIN_DTS_2.0', 'PT_DTS_1.0', 'AGE_AT_ADMISSION', 'BLD CREA_DTS_2.0', 'BUN_DTS_2.0', 'BUN_DTS_3.0', 'BLD CREA_DTS_1.0', 'BUN_DTS_1.0', 'ELIXHAUSER_SCORE']}
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 793.81ms

*** Performing cross validat

*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 46.092ms

*** Performing cross validation of 'RandomForestClassifier' classifier.
Number of splits: 10
*** Obtaining feature importances via classifier:
Selected features: 13
Running experiment with following parameters: 
{'target': 'AKI', 'cohort': 'MIMIC', 'test_size': 0.2, 'imputation_method': 'SimpleImputer', 'algorithm': 'RandomForest', 'sampling_method': None, 'scale_method': None, 'optimize_mode': False, 'num_features': 13, 'crossval': True, 'explainers': ['FeatContribExplainer'], 'features_to_select': ['HEMOGLOBIN_DTS_1.0', 'PLATELET COUNT_DTS_1.0', 'PTT_DTS_1.0', 'HEMATOCRIT_DTS_3.0', 'PT_DTS_1.0', 'BUN_DTS_1.0', 'AGE_AT_ADMISSION', 'BUN_DTS_3.0', 'BLD CREA_DTS_2.0', 'BLD CREA_DTS_3.0', 'BUN_DTS_2.0', 'BLD CREA_DTS_1.0', 'ELIXHAUSER_SCORE']}
*** Training of model 'RandomForestClassifier' started.
*** Training of classifier ready. Time elapsed: 998.231ms

*** Performing cross

In [8]:
''' store everything '''
if pickle(rfe_experiments, 'rfe_experiments.json'):
    print('Successfully saved.')

Pickling object...
Successfully saved.
